In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.cluster        import KMeans
from sklearn.neural_network import MLPRegressor
import kagglehub
from kagglehub import KaggleDatasetAdapter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor

from transformers import BertTokenizer, BertModel
from sklearn.decomposition import PCA
import torch
from torch.utils.data import Dataset, DataLoader

## Dataset Import and Feature Engineering

In [ ]:
file_path = "vehiclesclean.csv"
if os.path.isfile(file_path):
    df = pd.read_csv(file_path)
else:
    df = kagglehub.load_dataset(
      KaggleDatasetAdapter.PANDAS,
      "austinreese/craigslist-carstrucks-data",
      file_path
    )
    df = df.drop(columns=['id', 'region', 'url', 'region_url', 'VIN', 'image_url', 'description', 'county', 'lat', 'long', 'posting_date'])

In [ ]:
# print(len(df)) 426880 rows before dropping
df = df.dropna()
# print(len(df)) 79195 after dropping
df.head()

In [ ]:
numerics = ['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
categorical_columns = []
features = df.columns.values.tolist()

# Find categorical features
for col in features:
    if df[col].dtype in numerics: continue
    categorical_columns.append(col)

# Label encode categorical features
for col in categorical_columns:
    if col in df.columns:
        le = LabelEncoder()
        le.fit(list(df[col].astype(str).values))
        df[col] = le.transform(list(df[col].astype(str).values))

# Only use cars from last 15 years
df['year'] = (2025-df['year']).astype(int)
df = df[df['year'] < 15]

# Only using cars priced from 1-100k
df = df[df['price'] > 1000]
df = df[df['price'] < 150000]

# Binning odometer into groups of 5k
df['odometer'] = df['odometer'].astype(int)
df['odometer'] = df['odometer'] // 5000


def make_desc(row):
    return (
        f"This is a {row['condition']} {int(row['year'])} {row['manufacturer']} {row['model']}, "
        f"a {row['size']} sized {row['type']} with a {int(row['cylinders'])}-cylinder {row['fuel']} engine, "
        f"{row['transmission']} transmission, and {row['drive']} drive. "
        f"It has {int(row['odometer']):,} miles, holds a {row['title_status']} title in {row['state']}, "
        f"is painted {row['paint_color']}, predict its price."
    )

df['description'] = df.apply(make_desc, axis=1)

## Algorithms

In [ ]:
def LR(X_train, X_test, y_train, y_test): # Linear Regression
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled  = scaler.transform(X_test)

    model = LinearRegression()
    model.fit(X_train_scaled, y_train)

    y_pred = model.predict(X_test_scaled)

    return r2_score(y_test, y_pred), mean_squared_error(y_test, y_pred)

def PR(X_train, X_test, y_train, y_test): # Polynomial Regression
    poly = PolynomialFeatures(degree=2, include_bias=False)
    X_train_poly = poly.fit_transform(X_train)
    X_test_poly = poly.transform(X_test)

    model = LinearRegression()
    model.fit(X_train_poly, y_train)

    y_pred = model.predict(X_test_poly)

    return r2_score(y_test, y_pred), mean_squared_error(y_test, y_pred)

def DT(X_train, X_test, y_train, y_test): # Decision Tree
    model = DecisionTreeRegressor(random_state=0)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    return r2_score(y_test, y_pred), mean_squared_error(y_test, y_pred)

def NN(X_train, X_test, y_train, y_test): # Neural Network
    kmeans = KMeans(n_clusters=5, random_state=0)
    cluster_train = kmeans.fit_predict(X_train)
    cluster_test  = kmeans.predict(X_test)

    X_train_aug = np.hstack([X_train, cluster_train.reshape(-1,1)])
    X_test_aug  = np.hstack([X_test,  cluster_test.reshape( -1,1)])

    model = MLPRegressor(hidden_layer_sizes=(64, 32), max_iter=500, random_state=0)
    model.fit(X_train_aug, y_train)

    y_pred = model.predict(X_test_aug)

    return r2_score(y_test, y_pred), mean_squared_error(y_test, y_pred)

In [ ]:
X = df.drop(columns=['description', 'price'], axis=1)
y = df['price']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0
)
lr_r2, lr_mse = LR(X_train, X_test, y_train, y_test)
pr_r2, pr_mse = PR(X_train, X_test, y_train, y_test)
dt_r2, dt_mse = DT(X_train, X_test, y_train, y_test)
nn_r2, nn_mse = NN(X_train, X_test, y_train, y_test)

### BERT Encoding

In [ ]:
file_path = "dfbert.parquet"
if os.path.isfile(file_path):
    df = pd.read_parquet(file_path)
else:
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    class TextDataset(Dataset):
        def __init__(self, texts):
            self.texts = texts
            
        def __len__(self):
            return len(self.texts)
            
        def __getitem__(self, idx):
            return self.texts[idx]

    def process_batch(batch):
        encoded = tokenizer(
            batch,
            return_tensors='pt',
            padding=True,
            truncation=True,
            max_length=128
        )
        
        encoded = {k: v.to(device) for k, v in encoded.items()}

        with torch.no_grad():
            outputs = model(**encoded)
        
        return outputs.last_hidden_state[:, 0, :].cpu().numpy()

    dataset = TextDataset(df['description'].tolist())
    dataloader = DataLoader(dataset, batch_size=64, shuffle=False)

    embeddings = []
    for batch in dataloader:
        embeddings.append(process_batch(batch))
        
    embeddings_matrix = np.vstack(embeddings)

    pca = PCA(n_components=50, random_state=0)
    reduced_embeddings = pca.fit_transform(embeddings_matrix)

    pca_cols  = [f'bert_pca_{i}'  for i in range(reduced_embeddings.shape[1])]
    pca_df  = pd.DataFrame(reduced_embeddings, columns=pca_cols,  index=df.index)

    df = pd.concat([df, pca_df], axis=1)
    df.to_parquet('dfbert.parquet')
df.head()

In [ ]:
X = df.drop(columns=['description', 'price'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0
)
blr_r2, blr_mse = LR(X_train, X_test, y_train, y_test)
bpr_r2, bpr_mse = PR(X_train, X_test, y_train, y_test)
bdt_r2, bdt_mse = DT(X_train, X_test, y_train, y_test)
bnn_r2, bnn_mse = NN(X_train, X_test, y_train, y_test)

## Results and Plotting

In [ ]:
print("Standard")
print(f"R-squared (R²): \n\tLinear Regression: {lr_r2:.4f}\n\tPolynomial Regression: {pr_r2:.4f}\n\tDecision Tree: {dt_r2:.4f}\n\tNeural Network: {nn_r2:.4f}")
print()
print(f"Mean Squared Error (MSE): \n\tLinear Regression: {lr_mse:.2f}\n\tPolynomial Regression: {pr_mse:.2f}\n\tDecision Tree: {dt_mse:.2f}\n\tNeural Network: {nn_mse:.2f}")

print()
print()
print()

print("BERT")
print(f"R-squared (R²): \n\tLinear Regression: {blr_r2:.4f}\n\tPolynomial Regression: {bpr_r2:.4f}\n\tDecision Tree: {bdt_r2:.4f}\n\tNeural Network: {bnn_r2:.4f}")
print()
print(f"Mean Squared Error (MSE): \n\tLinear Regression: {blr_mse:.2f}\n\tPolynomial Regression: {bpr_mse:.2f}\n\tDecision Tree: {bdt_mse:.2f}\n\tNeural Network: {bnn_mse:.2f}")
